In [ ]:
# 코랩에서 light 버전 테스트
!pip install transformers==4.40.0 accelerate

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

In [ ]:
PROMPT = '''당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner.'''
# instruction = "문제인이 누구지?"
instruction = "윤석열은 누구지?"
# 로마에는 양반과 평민의 계급이 있었는데, 그 중에서도 평민 중에서도 지위가 높은 사람을 일컫는 것이 럼(量邑)입니다. 일반적으로 양반 중에서 평민에 속하는 사람이 럼이라고 불렸으며, 주로 포도청 종사관, 관부, 교리 등을 지냈습니다.
# 저는 문제를 파악하는 능력이 없습니다. 제가 제공하는 정보와 답변은 이전 지식과 자료를 기반으로 한 것입니다. 새로운 문제에 대한 답변을 위해서는 해당 분야의 전문 지식과 정보를 가진 전문가의 도움을 받는 것이 좋습니다.

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    repetition_penalty = 1.1
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))